In [1]:
# !pip3 install dgl
# !pip3 install torch 
# !pip3 install torchtext

import os
import re
import argparse
import pickle
import pandas as pd
import numpy as np
import scipy.sparse as ssp
import dgl
import torch
import torchtext
import networkx as nx
import sys 
sys.path.append('/Users/rebeccasalganik/Documents/School/2021-2022/Network Science/Capstone/PinSAGE-dgl/')
from builder import PandasGraphBuilder
from data_utils import *

Using backend: pytorch


In [2]:
directory = '/Users/rebeccasalganik/Documents/School/2021-2022/Network Science/Capstone/Spotify_Data/spotify_in_csv/'

In [3]:
data = pd.read_pickle(directory+'train_track_data.pkl')
print(len(data))

# data.dtypes

2262191


In [107]:
# d = data.drop_duplicates(subset=['track_uri']) 
# print(len(d))
# d.to_pickle(directory + "all_track_data.pkl")

2262191


In [4]:
tracks = data[['track_uri', 
       'rock', 'classical', 'classic rock', 'hip hop',
       'adult standards', 'rap', 'country rock', 'dance pop', 'mellow gold',
       'pop rap', 'gangster rap', 'folk rock', 'soft rock', 'pop',
       'album rock', 'alternative rock', 'pop rock', 'soul', 'latin',
       'southern hip hop']]
len(tracks)

2262191

In [5]:
plists = pd.read_csv(directory + 'train_playlists.csv', usecols = ['pid', 'name'], sep='\t')
plists.to_pickle(directory + "train_playlist_data.pkl")
len(plists)

1000000

In [6]:
interactions = pd.read_pickle(directory+'train_interaction_data.pkl')
interactions.head(2)

,pid,tid,track_uri
0,0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,123,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI


In [36]:
# subset = interactions.iloc[:30000, :].copy() 
# distinct_tracks = subset['track_uri'].unique()
# distinct_playlists = subset['pid'].unique()
# tracks = tracks[tracks['track_uri'].isin(distinct_tracks)]
# plists = plists[plists['pid'].isin(distinct_playlists)]

In [7]:
distinct_tracks = interactions['track_uri'].unique()
distinct_playlists = interactions['pid'].unique()
tracks = tracks[tracks['track_uri'].isin(distinct_tracks)]
plists = plists[plists['pid'].isin(distinct_playlists)]

In [8]:
t = tracks.drop_duplicates()
print(len(distinct_tracks), len(tracks), len(t))


2262190 2262190 2262190


In [9]:
graph_builder = PandasGraphBuilder()
graph_builder.add_entities(tracks, 'track_uri', 'track')
graph_builder.add_entities(plists, 'pid', 'playlist')
graph_builder.add_binary_relations(interactions, 'pid', 'track_uri', 'contains')
graph_builder.add_binary_relations(interactions, 'track_uri', 'pid', 'contained_by')

g = graph_builder.build()
g.number_of_nodes('playlist')
g.number_of_nodes('track')

2262190

In [12]:
genre_columns = tracks.columns.drop(['track_uri'])
interactions['present'] = list(1 for i in range(len(interactions)))
tracks.columns, plists.columns, genre_columns

(Index(['track_uri', 'rock', 'classical', 'classic rock', 'hip hop',
        'adult standards', 'rap', 'country rock', 'dance pop', 'mellow gold',
        'pop rap', 'gangster rap', 'folk rock', 'soft rock', 'pop',
        'album rock', 'alternative rock', 'pop rock', 'soul', 'latin',
        'southern hip hop'],
       dtype='object'),
 Index(['pid', 'name'], dtype='object'),
 Index(['rock', 'classical', 'classic rock', 'hip hop', 'adult standards',
        'rap', 'country rock', 'dance pop', 'mellow gold', 'pop rap',
        'gangster rap', 'folk rock', 'soft rock', 'pop', 'album rock',
        'alternative rock', 'pop rock', 'soul', 'latin', 'southern hip hop'],
       dtype='object'))

In [13]:
# g.nodes['playlist'].data['name'] = torch.LongTensor(plists['name'])

g.nodes['track'].data['genre'] = torch.FloatTensor(tracks[genre_columns].values)

g.edges['contains'].data['interaction'] = torch.LongTensor(interactions['present'].values)
g.edges['contained_by'].data['interaction'] = torch.LongTensor(interactions['present'].values)

In [24]:
def train_test_split_outer(df, user):
    df['train_mask'] = np.ones((len(df),), dtype=bool)
    df['val_mask'] = np.zeros((len(df),), dtype=bool)
    df['test_mask'] = np.zeros((len(df),), dtype=bool)
    df = dd.from_pandas(df, npartitions=10)
    def train_test_split(df):
        if df.shape[0] > 1:
            df.iloc[-1, -3] = False
            df.iloc[-1, -1] = True
        if df.shape[0] > 2:
            df.iloc[-2, -3] = False
            df.iloc[-2, -2] = True
        return df
    df = df.groupby(user, group_keys=False).apply(train_test_split).compute(scheduler='processes').sort_index()
    return df['train_mask'].to_numpy().nonzero()[0], \
           df['val_mask'].to_numpy().nonzero()[0], \
           df['test_mask'].to_numpy().nonzero()[0]

In [26]:
train_indices, val_indices, test_indices = train_test_split_outer(df = interactions, user = 'pid')

<ipython-input-24-003c50225fd3>:14: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df = df.groupby(user, group_keys=False).apply(train_test_split).compute(scheduler='processes').sort_index()


KeyboardInterrupt: 

In [ ]:
print("T: {}, V:{}, T:{}".format(train_indices, val_indices, test_indices))

In [ ]:
train_g = build_train_graph(g, train_indices, 'track', 'playlist', 'contains', 'contained_by')
assert train_g.out_degrees(etype='watched').min() > 0

In [ ]:
output_path = '/Users/rebeccasalganik/Documents/School/2021-2022/Network Science/Capstone/Spotify_Data/baby_dataset.pkl'
dataset = {
        'train-graph': train_g,
        'val-matrix': val_matrix,
        'test-matrix': test_matrix,
        'item-texts': None,
        'item-images': None,
        'user-type': 'playlist',
        'item-type': 'track',
        'user-to-item-type': 'contains',
        'item-to-user-type': 'contained_by',
        'timestamp-edge-column': 'timestamp'}

with open(output_path, 'wb') as f:
    pickle.dump(dataset, f)

In [ ]:
#### BELOW THIS LINE IS OLD CODE 

In [2]:
def load_csv(path): 
    data = pd.read_csv(path, sep='\t')
    return data 
 

In [3]:
track_p = '/Users/rebeccasalganik/Documents/School/2021-2022/Network Science/Capstone/Spotify_Data/spotify_in_csv/tracks.csv'
playlist_p = '/Users/rebeccasalganik/Documents/School/2021-2022/Network Science/Capstone/Spotify_Data/spotify_in_csv/train_playlists.csv'
interaction_p = '/Users/rebeccasalganik/Documents/School/2021-2022/Network Science/Capstone/Spotify_Data/spotify_in_csv/train_interactions.csv'

tracks = load_csv(track_p)
playlists = load_csv(playlist_p)
playlists = playlists.rename(columns={"name": "playlist_name"})
interactions = load_csv(interaction_p)
print(tracks.columns, playlists.columns, interactions.columns)

Index(['tid', 'arid', 'alid', 'track_uri', 'track_name', 'duration_ms'], dtype='object') Index(['pid', 'playlist_name', 'collaborative', 'modified_at', 'num_albums',
       'num_tracks', 'num_followers', 'num_tracks.1', 'num_edits',
       'duration_ms', 'num_artists', 'description'],
      dtype='object') Index(['pid', 'tid', 'pos'], dtype='object')


In [27]:
tracks_all = tracks[['tid', 'arid', 'alid', 'track_uri', 'track_name']]
playlists_all = playlists[['pid', 'playlist_name', 'num_albums', 'num_artists', 'num_followers']]
interactions_all = interactions[['pid', 'tid']]

In [28]:
baby_t = tracks_all.iloc[:5, :].copy()
baby_p = playlists_all.iloc[:5, :].copy()
baby_i = interactions_all.iloc[:50, :].copy()



In [6]:
joined_1 = pd.merge(baby_i, baby_t, how='right', on=["tid"], copy=True)
joined_2 = pd.merge(joined_1, baby_p, how="inner", on ='pid', copy=True)
joined_2 = joined_2.astype({'pid': 'int', 'tid': 'int'})
print(joined_2)


   pid  tid  arid  alid                             track_uri  \
0    0    0     0     0  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
1    0    1     1     1  spotify:track:6I9VzXrHxO9rA9A5euc8Ak   
2    0    2     2     2  spotify:track:0WqIKmW4BTrj3eJFmnCKMv   
3    0    3     3     3  spotify:track:1AWQoqb9bSvzTjaLralEkT   
4    0    4     4     4  spotify:track:1lzr43nnXAijIGYnCT8M8H   

                                   track_name playlist_name  num_albums  \
0  Lose Control (feat. Ciara & Fat Man Scoop)    Throwbacks          47   
1                                       Toxic    Throwbacks          47   
2                               Crazy In Love    Throwbacks          47   
3                              Rock Your Body    Throwbacks          47   
4                                It Wasn't Me    Throwbacks          47   

   num_artists  num_followers  
0           37              1  
1           37              1  
2           37              1  
3           37              1 

In [29]:
total_set = joined_2[['pid','tid']].copy()
track_nodes = baby_t[['tid']].copy()
playlist_nodes = baby_p[['pid']].copy()

sample_features = [0, 1,2, 3, 4]
track_nodes['sample_feats'] = sample_features
playlist_nodes['sample_feats'] = sample_features
total_set['in'] = [1,1,1,1,1]

print(total_set.head())



   pid  tid  in
0    0    0   1
1    0    1   1
2    0    2   1
3    0    3   1
4    0    4   1


In [49]:
distinct_tracks_in_interactions = total_set['tid'].unique()
distinct_plists_in_interactions = total_set['pid'].unique()
tracks = track_nodes[track_nodes['tid'].isin(distinct_tracks_in_interactions)]
plists = playlist_nodes[playlist_nodes['pid'].isin(distinct_plists_in_interactions)]

print(tracks.head())

   tid  sample_feats
0    0             0
1    1             1
2    2             2
3    3             3
4    4             4


In [51]:
graph_builder = PandasGraphBuilder()
graph_builder.add_entities(tracks, 'tid', 'track')
graph_builder.add_entities(plists, 'pid', 'playlist')
graph_builder.add_binary_relations(total_set, 'pid', 'tid', 'contains')
graph_builder.add_binary_relations(total_set, 'tid', 'pid', 'contained_by')

g = graph_builder.build()
g.number_of_nodes('playlist')
g.number_of_nodes('track')
# g.number_of_edges('contained_by') #shouldn't be 5 --> come back 

5

In [42]:
g.nodes['track'].data['track_feature'] = torch.LongTensor(tracks['sample_feats'])
# g.nodes['track'].data['track_name'] = torch.LongTensor(baby_t['track_name'].cat.codes.values)
# g.nodes['track'].data['artist'] = torch.LongTensor(users['age'].cat.codes.values)
# g.nodes['track'].data['musical'] = torch.LongTensor(users['occupation'].cat.codes.values)
# g.nodes['track'].data['album'] = torch.LongTensor(users['occupation'].cat.codes.values)

g.nodes['playlist'].data['playlist_feature'] = torch.LongTensor(plists['sample_feats'])

In [52]:
train_indices, val_indices, test_indices = train_test_split(df = total_set, user = 'pid')

<ipython-input-43-d74d31a762e4>:14: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df = df.groupby(user, group_keys=False).apply(train_test_split).compute(scheduler='processes').sort_index()


In [47]:
print("T: {}, V:{}, T:{}".format(train_indices, val_indices, test_indices))

T: [0 1 2], V:[3], T:[4]


In [78]:
def build_train_graph2(g, train_indices, utype, itype, etype, etype_rev):
    train_g = g.edge_subgraph(
        {etype: train_indices, etype_rev: train_indices},
        ) #relabel_nodes=False
    for ntype in g.ntypes:
        for col, data in g.nodes[ntype].data.items(): 
            train_g.nodes[ntype].data[col] = data
    for etype in g.etypes:
        for col, data in g.edges[etype].data.items():
            train_g.edges[etype].data[col] = data[train_g.edges[etype].data[dgl.EID]]
    print(train_g)
    return train_g

In [88]:
# train_g = build_train_graph(g, train_indices, 'track', 'playlist', 'contains', 'contained_by')
train_g = build_train_graph2(g, train_indices, 'track', 'playlist', 'contains', 'contained_by')
assert train_g.out_degrees(etype='contains').min() > 0

Graph(num_nodes={'playlist': 1, 'track': 3},
      num_edges={('playlist', 'contains', 'track'): 3, ('track', 'contained_by', 'playlist'): 3},
      metagraph=[('playlist', 'track', 'contains'), ('track', 'playlist', 'contained_by')])


In [94]:
val_matrix, test_matrix = build_val_test_matrix(g, val_indices, test_indices, 'playlist', 'track', 'contains')

In [95]:
output_path = '/Users/rebeccasalganik/Documents/School/2021-2022/Network Science/Capstone/Spotify_Data/baby_dataset.pkl'
dataset = {
        'train-graph': train_g,
        'val-matrix': val_matrix,
        'test-matrix': test_matrix,
#         'item-texts': movie_textual_dataset,
        'item-images': None,
        'user-type': 'playlist',
        'item-type': 'track',
        'user-to-item-type': 'contains',
        'item-to-user-type': 'contained_by',
        'timestamp-edge-column': 'timestamp'}

with open(output_path, 'wb') as f:
    pickle.dump(dataset, f)